In [1]:
from textrank4zh import TextRank4Sentence
from pyspark.sql.functions import udf

m1 = spark.read.option("multiLine", True).json("hdfs://master/tmp/data_hsu/m1/M1Data_spark.json")


import pandas as pd
m1_p = m1.toPandas()



In [2]:
m1_p["info"] = m1_p["info"].apply(lambda x : x.replace("\\xa0\\xa0","").replace("'",'"'))

import json
from json.decoder import JSONDecodeError
def good(info):
    try:
        info = json.loads(info)
    except JSONDecodeError:
        info = "bad"
    return info

m1_p["info"] = m1_p["info"].apply(good)
m1_p = m1_p[m1_p["info"] != "bad"]



In [3]:
import re
def reg_img(imgs):
    
    result = []
    for img in imgs:
        if not re.findall(r"^[https:]+",img):
            img = "https:" + img
        else:
            pass
        
        if re.findall(r"jpg$",img):
            result.append(img)
    return result

m1_p["img_1"] = m1_p["img"].apply(lambda x : json.loads(x.replace("'",'"')))


In [4]:
m1_p["img"] = m1_p["img_1"].apply(reg_img)

m1_p["article_hit"] = m1_p["article_hit"].apply(lambda x : int(x.replace("次","").replace(",","")))

m1_p = m1_p.drop(["author", "comment", "species", "img_1"],axis=1)

m1 = spark.createDataFrame(m1_p)


In [12]:
m1.rdd.getNumPartitions()
m1_new = m1.repartition(20)

In [13]:
m1_new.rdd.getNumPartitions()

20

In [14]:
m1_new.printSchema()

root
 |-- Name: string (nullable = true)
 |-- article_hit: long (nullable = true)
 |-- content: string (nullable = true)
 |-- date: string (nullable = true)
 |-- href: string (nullable = true)
 |-- img: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- info: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)
 |-- phone: long (nullable = true)
 |-- rate: long (nullable = true)
 |-- title: string (nullable = true)



# Textrank
```
json_schema = ArrayType(StructType([StructField('a', IntegerType(

), nullable=False), StructField('b', IntegerType(), nullable=False)]))

```

In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

tr4z = TextRank4Sentence()

def udf_tr4z(word):
    
    tr4z.analyze(text=word, lower=True, source="all_filters")
    result = []
    for setence in tr4z.get_key_sentences(num=3,sentence_min_len=6):
        result.append(setence.get("sentence"))
    return result

udf_tr = udf(udf_tr4z, returnType=ArrayType(StringType()))

m1_new = m1_new.withColumn("textrank", udf_tr(m1_new.content))



In [16]:
m1_new.coalesce(1).rdd.getNumPartitions()

1

In [17]:
m1_new.write.format('json').save("hdfs://master/tmp/data_hsu/m1/result2")

In [18]:
m1_toOne = spark.read.json("hdfs://master/tmp/data_hsu/m1/result2/")
m1_toOne.count()

1749

In [23]:
m1_toOne = m1_toOne.drop("content")

In [24]:
m1_toOne.coalesce(1).write.format('json').save("hdfs://master/tmp/data_hsu/m1/result_json")